In [1]:
# Projecte Final IT ACADEMY ENRIC SENA ALVAREZ


In [135]:
## Importo els paquets necessaris

from selenium import webdriver
import re
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import folium
import numpy as np
from sklearn.preprocessing import StandardScaler
from geopy.distance import distance
import matplotlib.pyplot as plt
from gmplot import gmplot
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

In [3]:
#Defineixo el driver que faré servir
driver =webdriver.Chrome("chromedriver")

# Agafo tots els links de les busquedes per municipis de les 4 provincies de Catalunya.
lleida='https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm'
barcelona='https://www.habitaclia.com/alquiler-vivienda-en-barcelona/listapoblaciones.htm'
tarragona='https://www.habitaclia.com/alquiler-vivienda-en-tarragona/listapoblaciones.htm'
girona='https://www.habitaclia.com/alquiler-vivienda-en-girona/listapoblaciones.htm'

llista_municipis=[lleida,barcelona,tarragona,girona]
# faig un doble bucle per tal de recorrer les 4 provincies i dintre de cada provincia, obtenir tots els links dels municipis que tenen pisos en lloguer a habitaclia.
links_municipis_list=[]
for i in range(len(llista_municipis)):
    driver.get(llista_municipis[i])
    lnks_municipis=driver.find_elements(By.TAG_NAME,'a')
    for lnk in lnks_municipis:
        links_municipis_list.append(lnk.get_attribute('href'))
    
links_municipis_list


['https://www.habitaclia.com/alquiler-vivienda-en-lleida/buscador.htm',
 'https://www.habitaclia.com/alquiler_temporada-vivienda-en-lleida/listapoblaciones.htm',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#A',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#B',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#C',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#D',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#E',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#F',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#G',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#H',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#I',
 'https://www.habitaclia.com/alquiler-vivienda-en-lleida/listapoblaciones.htm#J',
 'https://www.habi

In [4]:
# Netejaré els links ja que n'hi ha que no son de municipis i per tant no ens interessen.

llista_neta_municipis=list(filter(lambda x:'.com/alquiler' in x, links_municipis_list)) # Agafo els links que continguin el string: '.com/alquiler' per tal de depurar la llista
llista_neta_municipis=[x for x in llista_neta_municipis if 'listapoblaciones'not in x and 'alquiler.htm'not in x and 'buscador'not in x and 'oficinas'not in x] # Per acabar de depurar he detectat que s'han colat links que no interessen i els elimino seleccionant paraules que apareguin en aquests.
llista_neta_municipis

['https://www.habitaclia.com/alquiler-ager.htm',
 'https://www.habitaclia.com/alquiler-agramunt.htm',
 'https://www.habitaclia.com/alquiler-alamus.htm',
 'https://www.habitaclia.com/alquiler-albatarrec.htm',
 'https://www.habitaclia.com/alquiler-alcarras.htm',
 'https://www.habitaclia.com/alquiler-alguaire.htm',
 'https://www.habitaclia.com/alquiler-almacelles.htm',
 'https://www.habitaclia.com/alquiler-alpicat.htm',
 'https://www.habitaclia.com/alquiler-alt_aneu.htm',
 'https://www.habitaclia.com/alquiler-anglesola.htm',
 'https://www.habitaclia.com/alquiler-artesa_de_segre.htm',
 'https://www.habitaclia.com/alquiler-balaguer.htm',
 'https://www.habitaclia.com/alquiler-barbens.htm',
 'https://www.habitaclia.com/alquiler-bellpuig.htm',
 'https://www.habitaclia.com/alquiler-bellvis.htm',
 'https://www.habitaclia.com/alquiler-borges_blanques.htm',
 'https://www.habitaclia.com/alquiler-camarasa.htm',
 'https://www.habitaclia.com/alquiler-cervera.htm',
 'https://www.habitaclia.com/alquiler

In [5]:
links_list=[]
casa_list=[]
ubi_list=[]
caract_list=[]
preu_list=[]
for i in range(len(llista_neta_municipis)):
    # Agafo el link de cada pis
    driver.get(llista_neta_municipis[i])
    while True:
        #Agafo les cases que hi ha anunciades
        casa=driver.find_elements(By.CLASS_NAME, 'list-item-content')
        casa

        
        for e in range(len(casa)):
            casa_list.append(casa[e].text)
        


            # Agafo les ubicacions de cada casa
        ubi=driver.find_elements(By.CLASS_NAME,'list-item-location')


        for i in range(len(ubi)):
            ubi_list.append(ubi[i].text)
            

            # Agafo les característiques del pis
        caracteristiques=driver.find_elements(By.CLASS_NAME,'list-item-feature')


        for n in range(len(caracteristiques)):
            caract_list.append(caracteristiques[n].text)


            #Agafo el preu de cada pis
        preu=driver.find_elements(By.CLASS_NAME,'font-2')


        for i in range(len(preu)):
            preu_list.append(preu[i].text)



            # Agafo el link de cada pis
        lnks=driver.find_elements(By.TAG_NAME,'a')
        # traverse list
        
        for lnk in lnks:
            # get_attribute() to get all href
            links_list.append(lnk.get_attribute("href"))
    

        try:
            next_button = driver.find_element(By.XPATH,'//a[text()="Siguiente"]')
            next_button.click()
        except NoSuchElementException:
            break    


In [6]:
data = list(zip(casa_list[0:],ubi_list[0:],caract_list[0:],preu_list[0:]))
df =pd.DataFrame(data,columns=['Títol','Ubicació','Característiques','Preu'])


In [7]:
df[['Municipi','Barri']]=df['Ubicació'].str.split('-', expand=True,n=1)
df[['Superficie','Habitacions','Lavabo','Preum2']]=df['Característiques'].str.split('-',expand=True)    
df.head(10)     

,Títol,Ubicació,Característiques,Preu,Municipi,Barri,Superficie,Habitacions,Lavabo,Preum2
0,Alquiler Piso en C/ orient. Solvia inmobiliari...,Àger,"76m2 - 3 habitaciones - 1 baño - 10,53€/m2",800 €,Àger,None,76m2,3 habitaciones,1 baño,"10,53€/m2"
1,Alquiler Piso en Joan oro 4. Precioso piso en ...,Àger,"76m2 - 3 habitaciones - 1 baño - 6,32€/m2",480 €,Àger,None,76m2,3 habitaciones,1 baño,"6,32€/m2"
2,Alquiler Casa Castell\nAgramunt\n130m2 - 2 hab...,Agramunt,"130m2 - 2 habitaciones - 3 baños - 3,08€/m2",400 €,Agramunt,None,130m2,2 habitaciones,3 baños,"3,08€/m2"
3,Alquiler Apartamento Barcelona. Alamus apart. ...,Alamús (Els),"65m2 - 2 habitaciones - 2 baños - 6,62€/m2",430 €,Alamús (Els),None,65m2,2 habitaciones,2 baños,"6,62€/m2"
4,Alquiler Casa Carrer sudanell\nAlbatàrrec\n300...,Albatàrrec,"300m2 - 4 habitaciones - 3 baños - 2,50€/m2",750 €,Albatàrrec,None,300m2,4 habitaciones,3 baños,"2,50€/m2"
5,Alquiler Piso Sudanell\nAlbatàrrec\n63m2 - 1 h...,Albatàrrec,"63m2 - 1 habitacion - 1 baño - 7,14€/m2",450 €,Albatàrrec,None,63m2,1 habitacion,1 baño,"7,14€/m2"
6,Alquiler Apartamento en Era pona 10. Apartamen...,Albatàrrec,"35m2 - 1 habitacion - 1 baño - 10,86€/m2",380 €,Albatàrrec,None,35m2,1 habitacion,1 baño,"10,86€/m2"
7,Se encuentra en Montoliu de Lleida\nAlquiler C...,Montoliu de Lleida,"115m2 - 4 habitaciones - 1 baño - 6,52€/m2",750 €,Montoliu de Lleida,None,115m2,4 habitaciones,1 baño,"6,52€/m2"
8,Se encuentra en Lleida\nAlquiler Piso Sant fra...,Lleida - La Bordeta,"112m2 - 4 habitaciones - 2 baños - 5,80€/m2",650 €,Lleida,La Bordeta,112m2,4 habitaciones,2 baños,"5,80€/m2"
9,Se encuentra en Lleida\nAlquiler Piso en La Bo...,Lleida - La Bordeta,"93m2 - 4 habitaciones - 2 baños - 6,99€/m2",650 €,Lleida,La Bordeta,93m2,4 habitaciones,2 baños,"6,99€/m2"


In [8]:
copia=df # Faig una copia per si en un futur hi ha algún error, no haver de tornar a executar la cerca de informació

In [9]:
#Genero noves variables per tal de extreure només els numeros d'aquestes
df['num banys']=df['Lavabo'].str.extract(r'(\d+)\s+baño')
df['num superficie']=df['Superficie'].apply(lambda x: re.findall('\d+',x)[0] if re.findall('\d+', x) else None)
df['num habitacions'] = df['Habitacions'].str.extract(r'(\d+)\s+habitacion')
df['num preu']=df['Preu'].str.extract('(\d+)')

In [10]:
df.head(100)

,Títol,Ubicació,Característiques,Preu,Municipi,Barri,Superficie,Habitacions,Lavabo,Preum2,num banys,num superficie,num habitacions,num preu
0,Alquiler Piso en C/ orient. Solvia inmobiliari...,Àger,"76m2 - 3 habitaciones - 1 baño - 10,53€/m2",800 €,Àger,None,76m2,3 habitaciones,1 baño,"10,53€/m2",1,76,3,800
1,Alquiler Piso en Joan oro 4. Precioso piso en ...,Àger,"76m2 - 3 habitaciones - 1 baño - 6,32€/m2",480 €,Àger,None,76m2,3 habitaciones,1 baño,"6,32€/m2",1,76,3,480
2,Alquiler Casa Castell\nAgramunt\n130m2 - 2 hab...,Agramunt,"130m2 - 2 habitaciones - 3 baños - 3,08€/m2",400 €,Agramunt,None,130m2,2 habitaciones,3 baños,"3,08€/m2",3,130,2,400
3,Alquiler Apartamento Barcelona. Alamus apart. ...,Alamús (Els),"65m2 - 2 habitaciones - 2 baños - 6,62€/m2",430 €,Alamús (Els),None,65m2,2 habitaciones,2 baños,"6,62€/m2",2,65,2,430
4,Alquiler Casa Carrer sudanell\nAlbatàrrec\n300...,Albatàrrec,"300m2 - 4 habitaciones - 3 baños - 2,50€/m2",750 €,Albatàrrec,None,300m2,4 habitaciones,3 baños,"2,50€/m2",3,300,4,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Se encuentra en Lleida\nAlquiler Piso Carrer s...,Lleida - Centre Històric,"80m2 - 3 habitaciones - 2 baños - 7,95€/m2",636 €,Lleida,Centre Històric,80m2,3 habitaciones,2 baños,"7,95€/m2",2,80,3,636
96,Se encuentra en Lleida\nAlquiler Dúplex en Cen...,Lleida - Centre Històric,"168m2 - 5 habitaciones - 3 baños - 5,06€/m2",850 €,Lleida,Centre Històric,168m2,5 habitaciones,3 baños,"5,06€/m2",3,168,5,850
97,Se encuentra en Lleida\nAlquiler Piso Avinguda...,Lleida - Centre Històric,"55m2 - 2 habitaciones - 1 baño - 11,27€/m2",620 €,Lleida,Centre Històric,55m2,2 habitaciones,1 baño,"11,27€/m2",1,55,2,620
98,Se encuentra en Lleida\nAlquiler Apartamento R...,Lleida - Centre Històric,"80m2 - 2 habitaciones - 1 baño - 7,50€/m2",600 €,Lleida,Centre Històric,80m2,2 habitaciones,1 baño,"7,50€/m2",1,80,2,600


In [11]:
# Ara arreglaré els buits de barri ja que entenc que al ser municipis petits potser no tenen més d'un barri o bé així ho entén la web dels anuncis.
df['Barri']=df['Barri'].fillna(df['Municipi'])

In [7]:
# Només executar quan no s'executa la obtinguda de dades.

df2=pd.read_csv('dades_24-03.csv')
df2.head()

,Unnamed: 0,Títol,Ubicació,Característiques,Preu,Municipi,Barri,Superficie,Habitacions,Lavabo,Preum2,num banys,num superficie,num habitacions,num preu
0,0,Alquiler Piso en C/ orient. Solvia inmobiliari...,Àger,"76m2 - 3 habitaciones - 1 baño - 10,53€/m2",800 €,Àger,Àger,76m2,3 habitaciones,1 baño,"10,53€/m2",1.0,76.0,3.0,800
1,1,Alquiler Piso en Joan oro 4. Precioso piso en ...,Àger,"76m2 - 3 habitaciones - 1 baño - 6,32€/m2",480 €,Àger,Àger,76m2,3 habitaciones,1 baño,"6,32€/m2",1.0,76.0,3.0,480
2,2,Alquiler Casa Castell\nAgramunt\n130m2 - 2 hab...,Agramunt,"130m2 - 2 habitaciones - 3 baños - 3,08€/m2",400 €,Agramunt,Agramunt,130m2,2 habitaciones,3 baños,"3,08€/m2",3.0,130.0,2.0,400
3,3,Alquiler Apartamento Barcelona. Alamus apart. ...,Alamús (Els),"65m2 - 2 habitaciones - 2 baños - 6,62€/m2",430 €,Alamús (Els),Alamús (Els),65m2,2 habitaciones,2 baños,"6,62€/m2",2.0,65.0,2.0,430
4,4,Alquiler Casa Carrer sudanell\nAlbatàrrec\n300...,Albatàrrec,"300m2 - 4 habitaciones - 3 baños - 2,50€/m2",750 €,Albatàrrec,Albatàrrec,300m2,4 habitaciones,3 baños,"2,50€/m2",3.0,300.0,4.0,750


In [10]:
# Ara creo un nou dataframe amb la informació que m'interessa per fer les prediccions:
df=df2
df2=df.loc[:,['num preu','num habitacions','num superficie','num banys','Municipi','Barri']]

In [11]:
# Asigno els formats corresponents a cada variable:
cols_to_convert=['num preu','num habitacions','num superficie','num banys']
df2[cols_to_convert]=df2[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [12]:
#ara converteixo les dues columnes categòriques en factor.
cols_to_factor=['Municipi','Barri']
df2[cols_to_factor]=df2[cols_to_factor].astype('category')

In [18]:
df2.head(5)

,num preu,num habitacions,num superficie,num banys,Municipi,Barri
0,800,3.0,76.0,1.0,Àger,Àger
1,480,3.0,76.0,1.0,Àger,Àger
2,400,2.0,130.0,3.0,Agramunt,Agramunt
3,430,2.0,65.0,2.0,Alamús (Els),Alamús (Els)
4,750,4.0,300.0,3.0,Albatàrrec,Albatàrrec


In [21]:
df2.Municipi.value_counts

<bound method IndexOpsMixin.value_counts of 0                Àger
1                Àger
2            Agramunt
3        Alamús (Els)
4          Albatàrrec
             ...     
15311         Girona 
15312         Girona 
15313         Girona 
15314         Girona 
15315         Girona 
Name: Municipi, Length: 15316, dtype: category
Categories (463, object): ['Abrera', 'Agramunt', 'Aiguafreda', 'Aiguamúrcia', ..., 'Vinyols i els Arcs', 'Xàtiva', 'Àger', 'Òrrius']>

In [47]:
valors_unics = df2['Municipi'].unique()


In [15]:
# Localitzar els municipis
geolocator = Nominatim(user_agent="MyApp")
location = geolocator.geocode("Barcelona")
barcelona_lat = location.latitude
barcelona_long = location.longitude

# Funció que retorna la distància entre dos punts geogràfics
def calculate_distance(municipi):
    try:
        # Obtenim les coordenades del municipi actual
        municipi_location = geolocator.geocode(municipi)
        municipi_lat = municipi_location.latitude
        municipi_long = municipi_location.longitude

        # Calculem la distància entre Barcelona i el municipi actual
        distance = geodesic((barcelona_lat, barcelona_long), (municipi_lat, municipi_long)).km

        return (distance,municipi_lat,municipi_long)
    except:
        return (None, None, None)


In [ ]:
df[['distance', 'latitude', 'longitude']] = df['Municipi'].apply(calculate_distance)


In [58]:
distances
dist1=pd.DataFrame(distances)
dist1
dist1=dist1.transpose()
dist1 = dist1.drop(0, axis=1)


,0,1,2
Abrera,27.594762,41.520446,1.902413
Agramunt,100.560629,41.787669,1.098683
Aiguafreda,43.355835,41.769253,2.251716
Aiguamúrcia,68.782792,41.329339,1.358541
Alamús (Els),122.768532,41.614409,0.739704
...,...,...,...
Vilassar de Mar,22.535516,41.506404,2.391388
Vinyols i els Arcs,100.006598,41.113999,1.038725
Xàtiva,351.360138,38.988087,-0.520005
Àger,136.232402,42.000088,0.763225


In [68]:
barcelona_coord=(barcelona_lat,barcelona_long)
barcelona_coord

(41.3828939, 2.1774322)

In [73]:
dist1 = dist1.reset_index().rename(columns={'index': 'Municipi'})



,Municipi,1,2
0,Abrera,41.520446,1.902413
1,Agramunt,41.787669,1.098683
2,Aiguafreda,41.769253,2.251716
3,Aiguamúrcia,41.329339,1.358541
4,Alamús (Els),41.614409,0.739704
...,...,...,...
456,Vilassar de Mar,41.506404,2.391388
457,Vinyols i els Arcs,41.113999,1.038725
458,Xàtiva,38.988087,-0.520005
459,Àger,42.000088,0.763225


In [74]:
# Asigno els noms a les columnes de dist1
dist1.columns = ['Municipi', 'Lat', 'Long']

In [78]:
# Faig una funció per calcular la distancia a Barcelona a partir de les coordenades de cada municipi. 
def distance_to_barcelona(row):
    coords = (row['Lat'], row['Long'])
    return distance(coords, barcelona_coord).km

# Crea una nova columna al data frame amb la distància a Barcelona per a cada municipi
dist1['Distancia a Barcelona (km)'] = dist1.apply(distance_to_barcelona, axis=1)

In [81]:
# Junto els 2 dataframes df2 amb tota la info i dist1 amb la informació de la distancia.
df3 = pd.merge(df2, dist1, on='Municipi')

In [82]:
df3

,num preu,num habitacions,num superficie,num banys,Municipi,Barri,Lat,Long,Distancia a Barcelona (km)
0,800,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
1,480,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
2,480,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
3,500,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
4,400,2.0,130.0,3.0,Agramunt,Agramunt,41.787669,1.098683,100.560629
...,...,...,...,...,...,...,...,...,...
15221,3,3.0,120.0,2.0,Vidreres,Terrafortuna - Puig Ventós,41.788868,2.779277,67.465475
15222,1,4.0,280.0,NaN,Vidreres,Centre,41.788868,2.779277,67.465475
15223,3,4.0,100.0,2.0,Vilademuls,Vilademuls,42.138644,2.887915,102.648300
15224,4,7.0,450.0,4.0,Vilademuls,Vilademuls,42.138644,2.887915,102.648300


In [200]:
# Genero un mapa per veure les ubicacions i quant més vermell sigui el color, més elevat és el preu de cada localitat.
from folium.plugins import HeatMap
map_obj=folium.Map(location=barcelona_coord,zoom_start=10)
lats_longs=df3[['Lat','Long','num preu']].values.tolist()
HeatMap(lats_longs).add_to(map_obj)
map_obj
# Veig que hi ha coordenades un tant extranyes.

In [105]:
# Cambio el nom de la columna distancia a Barcelona per faclilitar el tractament de la columna.
df3.head(5)
df3 = df3.rename(columns={'Distancia a Barcelona (km)': 'distancia_barcelona'})
df3.head(5)


,num preu,num habitacions,num superficie,num banys,Municipi,Barri,Lat,Long,distancia_barcelona
0,800,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
1,480,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
2,480,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
3,500,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402
4,400,2.0,130.0,3.0,Agramunt,Agramunt,41.787669,1.098683,100.560629


In [122]:

# Funció per assignar les províncies en base a l'índex del municipi
def assigna_provincia(idx):
    if idx <= 963:
        return 'Lleida'
    elif idx <= 9876:
        return 'Barcelona'
    elif idx <= 12435:
        return 'Tarragona'
    elif idx <= 15226:
        return 'Girona'
    else:
        return None

# Aplica la funció a la columna "Municipi" i guarda el resultat a la nova columna "Provincia"
df3['Provincia'] = df3.index.map(assigna_provincia)

In [123]:
df3.head(5)

,num preu,num habitacions,num superficie,num banys,Municipi,Barri,Lat,Long,distancia_barcelona,geometry,Provincia
0,800,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402,POINT (42.00009 0.76322),Lleida
1,480,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402,POINT (42.00009 0.76322),Lleida
2,480,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402,POINT (42.00009 0.76322),Lleida
3,500,3.0,76.0,1.0,Àger,Àger,42.000088,0.763225,136.232402,POINT (42.00009 0.76322),Lleida
4,400,2.0,130.0,3.0,Agramunt,Agramunt,41.787669,1.098683,100.560629,POINT (41.78767 1.09868),Lleida
5,575,2.0,130.0,3.0,Agramunt,Agramunt,41.787669,1.098683,100.560629,POINT (41.78767 1.09868),Lleida


In [124]:
# ara creo la xarxa neuronal per predir.
X = df3.loc[:, ['num preu', 'num habitacions', 'num superficie', 'num banys', 'distancia_barcelona']].values
y = df3.loc[:, ['Provincia']].values


In [145]:
# creem una instància de LabelEncoder
le = LabelEncoder()

# etiquetem la variable Y
Y_encoded = le.fit_transform(y)
Y_encoded


c:\Users\enric\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [125]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=42)

In [173]:
model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 64)                384       
                                                                 
 dense_118 (Dense)           (None, 64)                4160      
                                                                 
 dense_119 (Dense)           (None, 64)                4160      
                                                                 
 dense_120 (Dense)           (None, 64)                4160      
                                                                 
 dense_121 (Dense)           (None, 64)                4160      
                                                                 
 dense_122 (Dense)           (None, 64)                4160      
                                                                 
 dense_123 (Dense)           (None, 64)              

In [174]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
381/381 [==============================] - 3s 2ms/step - loss: nan - accuracy: 0.5789 - val_loss: nan - val_accuracy: 0.6057
Epoch 2/50
381/381 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5803 - val_loss: nan - val_accuracy: 0.6057
Epoch 3/50
381/381 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5803 - val_loss: nan - val_accuracy: 0.6057
Epoch 4/50
381/381 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5803 - val_loss: nan - val_accuracy: 0.6057
Epoch 5/50
381/381 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5803 - val_loss: nan - val_accuracy: 0.6057
Epoch 6/50
381/381 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5803 - val_loss: nan - val_accuracy: 0.6057
Epoch 7/50
381/381 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5803 - val_loss: nan - val_accuracy: 0.6057
Epoch 8/50
381/381 [============================

In [161]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: nan
Test accuracy: 0.6057124137878418


In [158]:
preu_max = float(input("Introdueix el preu màxim que vols pagar: "))
dist_max = float(input("Introdueix la distància màxima a Barcelona: "))
habitacions=float(input("Introdueix el nombre d'habitacions que vols:"))
superficie=float(input("Introdueix quants m2 vols que tingui:"))
banys=float(input("Introdueix el nombre de banys que vols:"))

input_data = np.array([[preu_max, habitacions, superficie, banys, dist_max]])  
input_data_scaled = scaler.transform(input_data)
prediction = model.predict(input_data_scaled)
predicted_category = np.argmax(prediction, axis=1)[0] # Obté la categoria amb la probabilitat més alta
predicted_category_name = le.inverse_transform([predicted_category])[0] # Obté el nom de la categoria

print("La categoria predita és:", predicted_category_name)

1/1 [==============================] - 0s 17ms/step
La categoria predita és: Barcelona
